# Génération de la Frontière Efficiente de Markowitz

Ce notebook illustre l'utilisation de la méthode de scalarisation pour approximer la frontière efficiente d'un portefeuille d'actions du secteur Energie.


In [ ]:
import matplotlib.pyplot as plt
from scipy.constants import sigma
from scipy.optimize import minimize
import os

from level1.functions import *

# Charger les données
df = pd.read_csv('../datasets/Information_Technology.csv', index_col=0, parse_dates=True)

for file in os.listdir('../datasets/'):
    if file.endswith('.csv') and 'Information_Technology' not in file:
        temp_df = pd.read_csv(os.path.join('../datasets/', file), index_col=0, parse_dates=True)
        df = df.join(temp_df, how='inner')

In [ ]:
df

In [ ]:
df.info()

In [ ]:
# Calcul des rendements logarithmiques
returns = f_returns(df)

# Calcul des paramètres pour l'optimisation
mu = f_mu(returns)  # Annualisation (252 jours boursiers)
Sigma = f_sigma(returns)  # Annualisation de la matrice de covariance
num_assets = len(mu)

# Calcul des Paramètres d'Optimisation

Les rendements logarithmiques sont calculés comme :

$  r_t = \ln\left(\frac{P_t}{P_{t-1}}\right) $

Le vecteur des rendements moyens annualisés :

$ \mu = \frac{1}{T} \sum_{t=1}^T r_t \times 252 $

La matrice de covariance annualisée :

$ \Sigma = \frac{1}{T} \sum_{t=1}^T (r_t - \bar{r})(r_t - \bar{r})^T \times 252 $


# Méthode de résolution par scalarisation pour générer la frontière efficiente

Fonction rendement : $ F_1(w) = - (w^T \mu) $

Fonction risque : $ F_2(w) = w^T \Sigma w $

Fonction de cout de transaction : $ C(w) = \sum_{i=1}^{N} c_{prop} \cdot |w_i - w_i^{prev}| $

Fonction objectif scalarisée : $ F(w) = \lambda \cdot (w^T \Sigma w) - (1 - \lambda) \cdot (w^T \mu) $

## Use brute force to find different cardinality portfolios

In [ ]:
from itertools import combinations
from level2.cardinality_BF import optimize, worker

number_of_shares = 2 # Nombre d'actions dans le portefeuille

# Use the df-columns of the index in possibilities
lambdas = np.linspace(0, 1, 40)  # 100 points entre 0 et 1

In [ ]:
f_r, f_v, f_w = optimize(df, number_of_shares, lambdas, os.cpu_count()-2)

In [ ]:
import pickle
with open(f'frontier_data_{number_of_shares}.pkl', 'wb') as f:
    pickle.dump((f_r, f_v, f_w), f)

In [ ]:
def pareto_front(risque, rendement):
    # trier les portefeuilles par risque croissant
    idx = np.argsort(risque)
    risque_sorted = risque[idx]
    rendement_sorted = rendement[idx]

    # liste des points sur le front
    front_idx = [idx[0]]
    best_return = rendement_sorted[0]

    # on parcourt en augmentant le risque et on garde les meilleurs rendements
    for i in range(1, len(risque_sorted)):
        if rendement_sorted[i] > best_return:
            front_idx.append(idx[i])
            best_return = rendement_sorted[i]

    return np.array(front_idx)

risque = []
for e in f_v:
    risque.extend(e)
risque = np.array(risque)

rendement = []
for e in f_r:
    rendement.extend(e)
rendement = np.array(rendement)

weights = []
for e in f_w:
    weights.extend(e)
weights = np.array(weights)

indices_front = pareto_front(risque, rendement)
plt.scatter(risque, rendement, alpha=0.3)
plt.scatter(risque[indices_front], rendement[indices_front], color='red', s=40, label='Front de Pareto')
plt.legend()
plt.show()

frontier_returns = risque[indices_front]
frontier_volatilities = rendement[indices_front]
frontier_weights = weights[indices_front]


# Génération de la Frontière Efficiente

En faisant varier $ \lambda $ de 0 à 1, nous obtenons différents portefeuilles optimaux.


In [ ]:
print("Le portefeuille avec le rendement le plus élevé :")
max_return_index = np.argmax(frontier_returns)
print(f"Rendement : {frontier_returns[max_return_index]:.4f}, Volatilité : {frontier_volatilities[max_return_index]:.4f}")
weights = frontier_weights[max_return_index]
weights[weights < 1e-4] = 0  # Nettoyer les poids très faibles pour l'affichage
print(f"Actifs sélectionnés :")
for i, weight in enumerate(weights):
    if weight > 0:
        print(f"  {df.columns[i]} : {weight:.4f}")
#print(f"Poids : {weights}")

print("\nLe portefeuille avec le risque le plus faible :")
min_risk_index = np.argmin(frontier_volatilities)
print(f"Rendement : {frontier_returns[min_risk_index]:.4f}, Volatilité : {frontier_volatilities[min_risk_index]:.4f}")
weights = frontier_weights[min_risk_index]
weights[weights < 1e-4] = 0  # Nettoyer les poids très faibles pour l'affichage
print(f"Actifs sélectionnés :")
for i, weight in enumerate(weights):
    if weight > 0:
        print(f"  {df.columns[i]} : {weight:.4f}")
#print(f"Poids : {weights}")

In [ ]:
c = 0.01  # Coût proportionnel de transaction
w0 = np.array(num_assets * [1. / num_assets])  # Portefeuille

In [ ]:
import numpy as np
from pymoo.core.problem import Problem
from pymoo.algorithms.moo.nsga2 import NSGA2
from pymoo.operators.crossover.sbx import SBX
from pymoo.operators.mutation.pm import PM
from pymoo.operators.sampling.rnd import FloatRandomSampling
from pymoo.optimize import minimize
from pymoo.core.repair import Repair
from pymoo.visualization.scatter import Scatter
import matplotlib.pyplot as plt

class PortfolioRepair(Repair):
    def _do(self, problem, X, **kwargs):
        for i in range(len(X)):
            w = X[i]
            total = np.sum(w)
            if total != 1.0:
                X[i] = w / total
        return X

class PortfolioOptimizationProblem(Problem):
    def __init__(self, mu, cov_matrix, w0, c=0.001):
        n_var = len(mu)
        xl = np.zeros(n_var)
        xu = np.ones(n_var)
        super().__init__(n_var=n_var, n_obj=3, n_constr=0, xl=xl, xu=xu)
        self.mu = mu
        self.cov_matrix = cov_matrix
        self.w0 = w0
        self.c = c

    def _evaluate(self, X, out, *args, **kwargs):
        n_individuals = X.shape[0]
        returns = []
        risks = []
        costs = []

        for i in range(n_individuals):
            w = X[i, :]
            ret = w @ self.mu
            risk = w @ self.cov_matrix @ w
            cost = self.c * np.sum(np.abs(w - self.w0))
            returns.append(ret)
            risks.append(risk)
            costs.append(cost)

        returns = np.array(returns)
        risks = np.array(risks)
        costs = np.array(costs)

        # Gestion de la contrainte via pénalité
        constraint_violation = np.abs(np.sum(X, axis=1) - 1)
        penalty = 1000 * constraint_violation

        out["F"] = np.column_stack([
            -returns + penalty,
            risks + penalty,
            costs + penalty
        ])

# --- Configuration et lancement ---
n_assets = len(mu)
problem = PortfolioOptimizationProblem(mu, Sigma, w0, c)

algorithm = NSGA2(
    pop_size=100,
    n_offsprings=50,
    sampling=FloatRandomSampling(),
    crossover=SBX(prob=0.9, eta=15),
    mutation=PM(eta=20),
    repair=PortfolioRepair(),
    eliminate_duplicates=True
)

res = minimize(problem, algorithm, ('n_gen', 10_000), seed=1, verbose=True)

# --- Vérification et visualisation ---
if res.F is not None:
    optimal_weights = res.X
    objectives = res.F
    returns = -objectives[:, 0]  # On re-inverse le rendement
    risks = objectives[:, 1]
    costs = objectives[:, 2]

    # Visualisation 3D
    fig = plt.figure(figsize=(10, 7))
    ax = fig.add_subplot(111, projection='3d')
    sc = ax.scatter(risks, returns, costs, c=costs, cmap='viridis', alpha=0.7)
    ax.set_xlabel('Risque')
    ax.set_ylabel('Rendement')
    ax.set_zlabel('Coût')
    plt.colorbar(sc)
    plt.show()
else:
    print("L'optimisation n'a pas convergé. Aucune solution réalisable trouvée.")
    print("Message :", res.message)

est ce que on peut pas relaxer une conteainte de cardinalité par une pénalisation L1 ? Si oui sous quelle condition ?
Oui, il est possible de relaxer une contrainte de cardinalité en utilisant une pénalisation L1, mais cela dépend de certaines conditions. La contrainte de cardinalité impose une limite sur le nombre d'actifs non nuls dans un portefeuille, ce qui est une contrainte non convexe et difficile à gérer directement dans les problèmes d'optimisation. En revanche, la pénalisation L1 (ou régularisation Lasso) encourage la sparsité dans les solutions en ajoutant une pénalité proportionnelle à la somme des valeurs absolues des coefficients (poids des actifs dans le portefeuille).